In this simulation, real temperature data is used to drive a basic population dynamics model. The first thing to do is import and use the temperature file using `pandas`.

In [349]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/GJHSimmons/population-models/refs/heads/main/wx.csv", sep="\t")

df.head()

,MO,DA,YEAR,TMAX,TMIN,SOL,PRCP,RH,WIND
0,1,1,2000,18.9,2.1,160.9,0.0,14.4,2.1
1,1,2,2000,18.2,3.7,162.0,0.0,15.9,0.5
2,1,3,2000,14.5,2.0,127.3,0.0,10.7,3.1
3,1,4,2000,11.0,1.9,69.4,2.7,23.5,4.7
4,1,5,2000,7.3,-0.5,56.7,5.0,49.8,5.1


The data `df` contains temperature, solar radiation, precipitation, humidity and wind data for each day from 1/1/2000 to 31/12/2010. We want to allow the simulation to start from any day in this range. The following cell returns the row in the data which corresponds to a given day, month, and year.

In [350]:
# Set start date

start_day = 1
start_month = 6
start_year = 2003

start_index = df[(df.DA == start_day) & (df.MO == start_month) & (df.YEAR == start_year)].index[0]

For our simulation, the average temperature on a given day will be used to drive the model. The following function `T(t)` returns the average temperature on simulation day `t`, where day 0 is 1/6/2003, day 1 is 2/6/2003, and so on. 

In [351]:
def T(t):
    index = int(np.floor(t) + start_index)
    return 1/2 * (df["TMAX"][index] + df["TMIN"][index])

The following cell plots 2000 days of temperature data, starting 1/6/2003.

In [352]:
import plotly.express as px

t = np.linspace(0,2000, 10001)

temp = [T(day) for day in t]

fig = px.line(x=t, y=temp)

fig.show()

Now that we have the temperature data available and in a useable form, the definition of the simulation proceeds as normal. First, define the initial value problem to be simulated, in this case, $\frac{dy}{dt} = (b(t) - d(t))y$, $y(0) = 10$. The birth and death rate functions are defined in terms of the average temperature $T(t)$ as

$$ b(t) = \max(-0.00102 T(t)(T(t) - 25), 0) $$

and

$$ d(t) = 0.0005(T(t)^2 - 15.2T(t)) + 0.05 $$

respectively. 

In [353]:
def birth_rate(t):
    return max(-0.00102 * T(t) * (T(t) - 25), 0)

def death_rate(t):
    return 0.0005*(T(t)**2 - 15.2*T(t)) + 0.05

def f(t,y):
    return [(birth_rate(t) - death_rate(t))*y[0]]

y0 = [10]

Next, specify the time span for the simulation, and run the simulation using `solve_ivp`.

In [354]:
from scipy.integrate import solve_ivp

# Time span
t_start = 0
t_end = 2300

sol = solve_ivp(f, t_span = [t_start, t_end], y0=y0, dense_output=True)

Next, collect the simulation data into a `DataFrame` with a column for `time` and a column for each simulation variable.

In [355]:
import pandas as pd
import numpy as np

t = np.linspace(t_start, t_end, (t_end-t_start)*10 + 1)

df = pd.DataFrame({"time": t} | {f"y_{i+1}": sol.sol(t)[i] for i in range(len(y0))} | {'T': [T(day) for day in t], 'BR': [birth_rate(day) for day in t], 'DR': [death_rate(day) for day in t]})

Finally, plot the simulation.

In [356]:
import plotly.express as px

fig = px.line(data_frame=df, x=t, y=[f"y_{i+1}" for i in range(len(y0))])
fig.show()

You can also visualise how the birth and death rates change over time.

In [357]:
fig = px.line(data_frame=df, x=t, y=["BR", "DR"])
fig.show()